In [ ]:
#Pkg.clone("git://github.com/kbarbary/TimeIt.jl.git")
Pkg.add("Plots")
#Pkg.add("PyPlot")

In [ ]:
using TimeIt

In [ ]:

using Plots
pyplot(size = (300,300), legend = false)

### Julia is not an object oriented programming language

Julia does not use objects; that is, data structures where data and methods (functions) are bundled together. Instead you define structures that bundle data together and then separately write functions to operate on those structures.

### Modules

In the cell below I define a module with structures used in the simulation.

In [ ]:
module Diffusion

export AbstractPerson, Drunk, Field, loc

abstract type AbstractPerson end
 
struct Drunk <: AbstractPerson
    name::String 
    moves::Tuple
    
end
 
abstract type AbstractField end

struct loc
    x::Int
    y::Int
end

struct Field <: AbstractField
    drunks::Dict
    locations::Set{loc}
end    

end
using .Diffusion

### Functions

So here are the functions.

In [ ]:
function get_name(p::AbstractPerson)
    return p.name
end

function add(a::loc, b::loc)
    return loc(a.x+b.x, a.y+b.y)
end

function add(a::loc, b::Tuple)
    return loc(a.x+b[1], a.y+b[2])
end

function getdrunks(n; moves =((1,0),(0,1),(-1,0),(0,-1),(0,0)))
    return [Drunk(string(i), moves) for i in 1:n]
end

function create_field(n; dx=200, dy=200)
    drunks = Dict([(d,loc(rand(-dx:dx), rand(-dy:dy))) for d in getdrunks(n)]);
    return Field(drunks, Set{loc}(values(drunks)))
end

function move!(f::Field, d::Drunk)
    step = sample(d.moves)
    oldloc = f.drunks[d]
    newloc = add(oldloc, step)
    if ! in(newloc, f.locations)
        delete!(f.locations, oldloc)
        f.drunks[d] = newloc
        push!(f.locations, newloc)
    end
    return nothing
        
end  

function move_drunks!(f)
    for d in keys(f.drunks)
        move!(f, d)
    end
end


function run_simulation(f, numsteps; dx=200, dy=200)
    # f = create_field(numdrunks; dx=dx, dy=dy)
    plot_field(f, color="blue")
    for n in 1:numsteps
        move_drunks!(f)
    end
    plot_field(f, color="red")
    return f
end  


function sample(x)
    return x[rand(1:length(x))]
end
   

In [ ]:
function plot_field(f; xlim=(-800,800), ylim=(-800,800), color="red")
    _keys = keys(f.drunks)
    xs = [f.drunks[k].x for k in _keys]
    ys = [f.drunks[k].y for k in _keys]
    scatter(xs, ys, color="red", xlim=xlim, ylim=ylim)
end

In [ ]:
f = create_field(200);
plot_field(f)

In [ ]:
t = @timeit run_simulation(f, 100000);
print(t)


In [ ]:
plot_field(f)